In [ ]:
using Plots
using QuadGK
using NLopt
#using SpecialFunctions
using NLsolve
using Statistics
#include("../scaling_CGS.jl")
#include("../scaling_HL.jl")

include("../code_to_real.jl")

In [ ]:
#---------- Double current sheet setup ----------
base = BaseUnits("CGS")

electron_proton_mass_ratio = 25.0

electron_mass_scaling = (base.m_p / base.m_e) / electron_proton_mass_ratio
charge_scaling = 1e0 #1e-5
eps_0_scaling = 1e0 #1e2
mu_0_scaling = 1e0

scale_base_units(base, eps_0_scaling, mu_0_scaling, electron_mass_scaling, charge_scaling)


length = 1e0 #720.0847332647692
number_density= 1e9 #6.44179e10
B_flux= 50 #22.222
temperature   = 1e6
temperature_e = temperature
temperature_p = temperature



mu = ( (base.m_e + base.m_p)  / 2. ) /  base.m_u# assuming only protons and electrons
mass_density = number_density * 2 * mu * base.m_u
e_plasma = base.e * sqrt(4 * π * number_density / base.m_e)



#----------- These always sets the following code units --------------
#            rho=1, c=1, e_plasma=2pi
mass_density_scale = mass_density
time_scale         = 2 * pi / e_plasma
length_scale       = base.c * time_scale#
temperature_scale  = 1e0
#----------------------------------------------------------------------

length = length_scale


scaling = ScalingCGS(base,
                number_density, length, temperature, temperature_e, temperature_p, B_flux,
                length_scale, mass_density_scale, time_scale, temperature_scale)
ds = 4 / (32 * 32 )
per_cell = 32

set_macro_particle_weights(scaling, ds, per_cell);
#print_all_CGS(scaling)
print_basic_info(scaling)

In [ ]:
c_code = scaling.c_code
B_code = scaling.B_flux_code
p_code = scaling.pressure_code
rho_code = scaling.mass_density_code
debye_code = scaling.debye_len_code



c_u = code_units(c_code, B_code, p_code, rho_code, debye_code)
r_u = known_real_units(scaling.number_density_real)
eps_0_scaling = 1e0
charge_scaling = 1e0
electron_proton_mass_ratio = scaling.m_p_real / scaling.m_e_real

r_s = ratios_and_scaling(electron_proton_mass_ratio, eps_0_scaling, charge_scaling)


new_scaling = find_real_units_CGS(c_u, r_u, r_s)
print_basic_info(new_scaling)
print_all_CGS(new_scaling)

In [ ]:
c_code = scaling.c_code
B_code = scaling.B_flux_code / sqrt(4 * pi)
p_code = scaling.pressure_code
rho_code = scaling.mass_density_code
debye_code = scaling.debye_len_code 



c_u = code_units(c_code, B_code, p_code, rho_code, debye_code)
r_u = known_real_units(scaling.number_density_real)
eps_0_scaling = 1e0
charge_scaling = 1e0
electron_proton_mass_ratio = scaling.m_p_real / scaling.m_e_real

r_s = ratios_and_scaling(electron_proton_mass_ratio, eps_0_scaling, charge_scaling)


new_scaling = find_real_units_HL(c_u, r_u, r_s)
print_basic_info(new_scaling)
print_all_HL(new_scaling)

# Equations used to calculate convsersion

### Constants
- $c_{code} = $ speed of light in code values
- $B_{code} = $ Magnetic field strength in code values
- $P_{code} = $ pressure in code values
- $\rho_{code} = $ mass density in code values
---
- $c_{real} = $ speed of light in real units
- $B_{real} = $ Magnetic field strength in real units
- $P_{real} = $ pressure in real values
- $\rho_{real} = $ mass density in real units
---
- $t_s = $ time scaling
- $l_s = $ length scaling
- $m_s = $ mass scaling
- $\rho_s = \frac{m_s}{l_s^3}$ mass density scaling
 
 
### Basic equations

1)
$$ c_{code} = c_{real} \frac{t_s}{l_s}$$
2)
$$ B_{code} = B_{real} \frac{\sqrt{m_s}}{\sqrt{l_s} t_s}$$
3)
$$ P_{code} = P_{real} \frac{m_s}{l_s t_s}$$
4)
$$ \rho_{code} = \rho_{real} \frac{1}{\rho_s} = \rho_{real} \frac{l_s^3}{m_s}$$

### Combinations and reformulation of basic equations

Restructuring 1)

5)
$$ l_s = \frac{c_{real}}{c_{code}} t_s$$


Combining equation 2 and 3 yields:

6) 
$$\frac{P_{real}}{P_{code}} = \left( \frac{B_{real}}{B_{code}} \right)^2$$




## Extracting real values

### Extracting temperature

First need to equation for the electron thermal velocity
$$v_{t-e-real} = \sqrt{\frac{k_B * T}{m_e}} $$
This can also written as
$$v_{t-e-real} =  \sqrt{\frac{p_e}{\rho_e}}$$
where $p_e$ and $\rho_e$ is the electron pressure and mass density. 
So we have 
$$ \sqrt{\frac{p_e}{\rho_e}} = \sqrt{\frac{k_B * T}{m_e}}$$


Now, assuming that we have $P_e$ and $\rho_e$ in code units. We then need to scale the real units of $k_B$, $T$, and $m_e$ to code units for the equation to match: 

$$ \sqrt{\frac{p_e}{\rho_e}} = \sqrt{\frac{k_B \frac{t_s^2 T_s}{m_s l_s^2} * T \frac{1}{T_s}}{m_e \frac{1}{m_s}}}$$
where $T_s$ is not the temperature scaling - note however that this has no effect and we can leave it out. 
We can rewrite this to 
$$ \sqrt{\frac{p_e}{\rho_e}} = \sqrt{\frac{k_B * T}{m_e}} \frac{t_s}{l_s}$$
And isolating the temperature we can get:

$$T = \frac{P_e}{\rho_e} \frac{m_e}{k_B} \left( \frac{l_s}{t_s} \right)^2$$
Looking at equation 1) we can rewrite this to 
$$T = \frac{P_e}{\rho_e} \frac{m_e}{k_B} \left( \frac{c_{real}}{c_{code}} \right)^2$$

Finally, we can rewrite this in terms of the total plasma density and pressure as:


7)
$$T = \frac{P_{code}}{\rho_{code}} \frac{\mu m_u}{k_B} \left( \frac{c_{real}}{c_{code}} \right)^2$$

where $m_u$ is the elementary mass in real units, and $k_B$ is Botlzmanns constant in real units

----
### Extracting real pressure

Extracting the real pressure is now rather straight forward.

8)
$$P_{real} =  \frac{\rho_{real}}{\mu * m_u} * k_B * T  $$

---
### Finding the real magnetic field strength 
We can then use equation 6) to find the magnetic field strength:

9)
$$B_{real} = B_{code} \sqrt{\frac{P_{real}}{P_{code}}}$$
---
### Finding length scale

To find the length scale we need one length in code units.
For this, I have used the Debye length:

$$\lambda_D = \frac{v_{t-e}}{\omega_e} = \frac{\sqrt{\frac{k_B T}{m_e}}}{q_e \sqrt{\frac{4 \pi n_e}{m_e}}} = \frac{\sqrt{k_B T}}{q_e \sqrt{4 \pi n_e}} $$

Similar to before, we insert the debye length in code units, but use other variables in real units and insert the proper scaling factors:

$$\lambda_D = \frac{\sqrt{k_B \frac{t_s^2 T_s}{m_s l_s^2} * T \frac{1}{T_s}}}{q_e \frac{t_s}{l_s^{3/2} \sqrt{m_s}} \sqrt{4 \pi n_e l_s^3}} $$
Here, we can isolate the length scale as:

10)
$$l_s = \frac{1}{q_e \lambda_D} \sqrt{\frac{k_B T}{4 \pi n_e}}$$
Where again, $q_e$ is the elemetary charge in real units, $k_B$ is in real units, $n_e$ is in real units, $T$ is in real units, and $\lambda_D$ is in code units. 

---
### Finding the time scale

With the length scale found we easily find the time scale looking back at equation 1)

11)
$$t_s = l_s \frac{c_{code}}{c_{real}} $$

## Important comments

Note that if any physical variables are "fudged" this must be applied to the "real physical unit" in the above equations. For example, increasing the mass of the electron will cause equation 7) to give a higher temperature. Likewise, reducing the speed of light will yield a smaller temperature. 

Changing the elementary mass will change the length scale by an equal factor


### Additional comment
Depending on wheter to use Heaviside or Gaussian units, the equations will slightly change.
For Heaviside, the factor $4 \pi$ when finding length scale should be omitted. 

If units are given in Heaviside, but you are using Gaussian, you may need to scale magnetic field with $B_{CGS} = B_H * \sqrt{4 \pi}$ and the debye length by $\lambda_{D-CGS} = \lambda_{D-H} / (4 \pi )$